In [ ]:
import sys
import pathlib as pl
import math

import geopandas as gpd
import shapely.geometry as sg
import matplotlib.pyplot as plt

sys.path.append("../")

import geohexgrid as ghg


%load_ext autoreload
%autoreload 2

DATA_DIR = pl.Path("../tests/data")
NZTM = "epsg:2193"  # New Zealand Transverse Mercator CRS
WGS84 = "epsg:4326"

In [ ]:
# make_grid_points and make_grid

nrows, ncols, R = 3, 5, 1
ox, oy = -1, 2
X, Y = ghg.make_grid_points(nrows, ncols, R, ox, oy)
display(X)
display(Y)
grid = ghg.make_grid(nrows, ncols, R, ox, oy)
display(grid.head())

fig, ax = plt.subplots()
ax.scatter(X, Y)
#plt.gca().set_aspect('equal')
grid.plot(ax=ax, color="white", edgecolor="blue", alpha=0.5, aspect="equal")


In [ ]:
grid = ghg.make_grid(2, 3, R=1, ox=2, oy=1)
display(grid)
grid["cell_id"].tolist()

p = grid.unary_union.boundary
display(p, p.is_ring)

p1 = grid["geometry"].iat[0].buffer(-0.001)
p2 = grid["geometry"].iat[1]
q = p1.union(p2).boundary

display(q, q.is_ring)


In [ ]:
# make_grid_from_bounds

rect = gpd.GeoDataFrame(geometry=[sg.Polygon([(2.1, -1), (4.9, -1), (4.9, 1.9), (2.1, 1.9)])])
R = 1
grid = ghg.make_grid_from_bounds(*rect.total_bounds, R=R, ox=0, oy=0)
display(grid.head())
print(grid["cell_id"].tolist())

base = rect.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
shape = gpd.GeoDataFrame(geometry=[sg.Polygon([(1, -1), (3, 1), (0, 3)])])
grid = ghg.make_grid_from_gdf(shape, R=1, ox=0, oy=0)

display(grid.head())
print(grid["cell_id"].tolist())

base = shape.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
# make_grid_from_gdf

shapes = gpd.read_file(DATA_DIR / "shapes.geojson").to_crs(NZTM)
R = 900
grid = ghg.make_grid_from_gdf(shapes, R=R)
display(grid.head())

base = shapes.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)


In [ ]:
# make_grid_from_gdf

shapes = gpd.read_file(DATA_DIR / "shapes.geojson").to_crs(NZTM)
R = 900
grid = ghg.make_grid_from_gdf(shapes, R=R)
display(grid.head())

base = shapes.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)


In [ ]:
shapes = gpd.read_file(DATA_DIR / "shapes.geojson").to_crs(NZTM)
base = shapes.plot(color='red', figsize=(20, 20), aspect="equal")
ghs = ghg.GeoHexSystem(NZTM, 250, 0, 0)

# Grid 1 in black
grid1 = ghs.grid_from_gdf(shapes)
grid1.plot(ax=base, color='white', edgecolor="black", alpha=0.1)

# Grid 2 (intersected with shapes) in blue
grid2 = ghs.grid_from_gdf(shapes, intersect=True)
grid2.plot(ax=base, color='white', edgecolor="blue", alpha=0.5)

# Try again in WGS84 with larger cells
shapes = gpd.read_file(DATA_DIR / "shapes.geojson")
base = shapes.plot(color='red', figsize=(20, 20), aspect="equal")
ghs = ghg.GeoHexSystem(WGS84, 0.01, 0, 0)

# Grid 1 in black
grid1 = ghs.grid_from_gdf(shapes)
grid1.plot(ax=base, color='white', edgecolor="black", alpha=0.1)
display(grid1)

# Grid 2 (intersected with shapes) in blue
grid2 = ghs.grid_from_gdf(shapes, intersect=True)
grid2.plot(ax=base, color='white', edgecolor="blue", alpha=0.5)

# Grid 3 (clipped to shapes) in blue
grid3 = ghs.grid_from_gdf(shapes, clip=True)
#grid3.plot(ax=base, color='white', edgecolor="blue", alpha=0.5)
display(grid3)

In [ ]:
# Load New Zealand's territorial authorities projected in EPGS 2193 (NZTM)
shapes = gpd.read_file(DATA_DIR / "nz_tas.gpkg")
display(shapes.crs)

# Generate a minimal cover of it with hexagons of circumradius 10 kilometres
%time grid = ghg.make_grid(shapes, 10_000, intersect=True) 

base = shapes.plot(color='red', figsize=(20, 20), aspect="equal")
grid.plot(ax=base, color='white', edgecolor="blue", alpha=0.5)
